In [16]:
# NGSolve Libraries
from netgen.geom2d import unit_square
from ngsolve import *
from ngsolve.webgui import Draw # para jupyter
#import netgen.gui
from netgen.occ import *
from netgen.csg import *
from netgen.geom2d import SplineGeometry
import sys
sys.path.insert(0,"../")
import problems
import numpy as np
import params
import pickle
from time import time
from scipy.optimize import minimize_scalar



In [17]:
## Check positivity
def check_pos_gfu(gfu,mesh):
    vertices =( (0,0), (90,0))
    m1 = mesh(vertices[0])
    m2 = mesh(vertices[1])
    print(gfu(m1),gfu(m2))



In [18]:
## get problem parameters and geometry
problem = problems.problem2

phi0 = problem[0]['phi0']
chi = problem[0]['chi']
G_target = problem[0]['G']
geom = "../"+problem[1]
dim = problem[0]['dim']
BC = problem[2]
name = problem[-1]
h = 1
ord = 3
ord_phi = ord - 3
N = params.N
KBTV = params.KBTV
form = "Functional" # EDP //functional

rho = 1.23e-6


phi = lambda J: phi0/J

G = Parameter(G_target)

## Generate mesh and geometry ### add parallel stuff
def mesher(geom, h):
    if ".stp" in geom:
        geo = OCCGeometry(geom)
    else:
        geo = pickle.load(open(geom, "rb"))

    mesh = Mesh(geo.GenerateMesh(maxh=h))
    return mesh
mesh = mesher(geom, h) 




def Gel_energy_functional(F,u):
    g = 9.81
    J = Det(F)
   
    H = (J - phi0)*log(1-phi0/J)  + phi0 * chi*(1-phi0/J)
    # calculate C
    # min (G/2) * (3 lam**2) + KBTV * H(lam**3):  lam > 1
    Hf = lambda lam: (lam-phi0)*np.log(1-phi0/lam) + phi0*chi*(1-phi0/lam)

    fun = lambda lam: (G.Get()/2)*(3*lam**2) + KBTV*Hf(lam)
    res = minimize_scalar(fun, bounds=(1, 10), method='bounded')
    C = res.fun
    return 0.5*(G)* Trace(F.trans*F ) + H*KBTV + rho*g*u[1] - C

## Generate spaces and forms
"""
To bond the gel go to geometries and describe the bonding
face there, not here.
"""
if BC["dir_cond"] == "faces":
    fesu = VectorH1(mesh, order=ord, dirichlet = BC["DIR_FACES"])
elif BC["dir_cond"] == "components":
    fesu = VectorH1(mesh, order=ord, dirichletx = BC["x"], dirichlety = BC["y"], dirichletz = BC["z"])

fesphi = L2(mesh, order = ord_phi)

u, v = fesu.TnT()
delta, vphi = fesphi.TnT()
# Assemble all bilinear forms

## Assemble forms
alpha = Parameter(1)
psih = GridFunction(fesphi)
psih.Set(0)

eps_i = 1e-3
uk = GridFunction(fesu)

psik = GridFunction(fesphi)
psik.Set(0)
eps = 1e-6


obs = CF(-z)    


def Assembler_A(fesu):
    BF = BilinearForm(fesu, symmetric=False)
    F = Id(2)+ grad(u)
    BF += Variation(alpha * Gel_energy_functional(F,u).Compile()*dx)
    return BF


def Assembler_B(trialspace= fesphi, testspace=fesu):
    BF = BilinearForm(trialspace, testspace)
    BF += delta * v[1]*dx
    # BF += u[1]*vphi*dx??
    return BF

def Assembler_C(fesphi,psih):
    BF = BilinearForm(fesphi)
    if ord_phi == 0:
        BF+= -delta * exp(psih) * vphi * dx -eps * (delta * vphi * dx)
    else:
        BF+= -delta * exp(psih) * vphi * dx -eps * (grad(delta) * grad(vphi) * dx)
    return BF

def Assembler_L1(psih, psik, fes):
    LF = LinearForm(fes)
    LF += -(psik - psih)*v[1]*dx
    return LF
    
def Assembler_L2(fesphi):
    LF = LinearForm(fesphi)
    LF += -(obs + exp(psih))*vphi*dx
    return LF
    

## A###
A = Assembler_A(fesu)
### B ####
B = Assembler_B(trialspace=fesphi, testspace=fesu)
B.Assemble()
## C ###
C = Assembler_C(fesphi,psih)

## L1 ###
L1 = Assembler_L1(psih,psik,fesu)

## L2 ###
L2 = Assembler_L2(fesphi)


# define phi as the 0 function 

gfu = GridFunction(fesu)
scenes = GridFunction(fesu, multidim = 0)

In [19]:
Draw(mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [20]:
import scipy as sp
max_PG_it = 30
max_iter_newton = 50
max_iter_qnewton = 50
newton_damp = 0.5
softening_n = 20

tol_newton = 1e-6
tol_QN = 1e-6
tol_PG = 1e-6
tol_delta = 1e-6
gammas = np.flip(np.linspace(G_target, G_target*20 ,softening_n))
first_bit = gammas[:-1]
second_bit = np.flip(np.linspace(G_target,gammas[-2], 10))
gammas = np.concatenate((first_bit,second_bit))



for num, load in enumerate(gammas): # softening
    G.Set(load) #incremental softening
    print("Softening step:",num, "Load:",load)

    n = 0
    while n < max_iter_newton: # Proximal Galerkin
        print("Newton it:",n)
        u_n = GridFunction(fesu)
        u_n.vec.data = gfu.vec
        res_n = gfu.vec.CreateVector()
        A.Apply(u_n.vec,res_n)
        A.AssembleLinearization(u_n.vec)
        #A_inv = A.mat.Inverse(freedofs= fesu.FreeDofs())
        
        k = 0
        while k < max_PG_it: #PG
            print("PG it:",k)
            alpha.Set(2**k)
            psik.vec.data = psih.vec
            if k != 0:
                uk.vec.data = gfu.vec # update u_k to compare later
            u_it = GridFunction(fesu)
            d_it = GridFunction(fesphi)
            for it in range(max_iter_qnewton): #QN
                print("QN it:",it)
                with TaskManager():

                    L1 = Assembler_L1(psih,psik,fesu)
                    L1.Assemble()
                    L2 = Assembler_L2(fesphi)
                    L2.Assemble()
                    C = Assembler_C(fesphi,psih)
                    C.Assemble()
                # get scipy sparse matrix
                A_rows,A_cols,A_vals = A.mat.COO()
                B_rows,B_cols,B_vals = B.mat.COO()
                C_rows,C_cols,C_vals = C.mat.COO()

                # get sparse matrix
                A_mat = sp.sparse.csr_matrix((A_vals, (A_rows, A_cols)), shape=A.mat.shape)
                B_mat = sp.sparse.csr_matrix((B_vals, (B_rows, B_cols)), shape=B.mat.shape)
                C_mat = sp.sparse.csr_matrix((C_vals, (C_rows, C_cols)), shape=C.mat.shape)
                C_inv = sp.sparse.linalg.inv(C_mat)
                A_inv = sp.sparse.linalg.inv(A_mat)
                # get vectors in numpy form
                L1_vec = L1.vec.FV().NumPy()
                L2_vec = L2.vec.FV().NumPy()
                res_n_vec = res_n.FV().NumPy()
                # schur complement for u_it
                F = B_mat @ C_inv @ B_mat.T
                # get schur complement inverse
                # schur_u = A_inv - F
                schur_u_inv = sp.sparse.linalg.inv(A_mat - F)

                schur_delta_inv = sp.sparse.linalg.inv(C_mat - B_mat.T @ A_inv @ B_mat)
                u_vec = schur_u_inv @ (L1_vec + res_n_vec - B_mat @ C_inv @ L2_vec) # solve for u_it
                delta_vec = schur_delta_inv @ (L2_vec - B_mat.T @ A_inv @ L1_vec) # solve for delta

                # update grid function
                u_it.vec.FV().NumPy()[:] = u_vec
                d_it.vec.FV().NumPy()[:] = delta_vec
                qn_val = Integrate(d_it**2, mesh)
                print("QN val:",qn_val, tol_QN, qn_val < tol_QN)
                if abs(qn_val) < tol_QN:
                    print("QN converged")
                    break
                # update newton residual
            
                psih.vec.data = psih.vec + d_it.vec
            pg_val = Integrate((u_it-uk)**2, mesh)
            print("PG val:",pg_val, tol_PG, pg_val < tol_PG)
            if pg_val < tol_PG:
                print("PG converged")
                break
        # newton update
        
        gfu.vec.data = gfu.vec + newton_damp*u_it.vec
        newton_val = Integrate((gfu-u_n)**2, mesh)

Softening step: 0 Load: 2.6
Newton it: 0
PG it: 0
QN it: 0


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:597: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/scipy/sparse/linalg/_matfuncs.py:76: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  Ainv = spsolve(A, I)


QN val: 7.067807975607855e-28 1e-06 True
QN converged
PG val: 984.7188454767038 1e-06 False
PG it: 0
QN it: 0
QN val: 7.067807975607855e-28 1e-06 True
QN converged
PG val: 984.7188454767038 1e-06 False
PG it: 0
QN it: 0


KeyboardInterrupt: 

In [ ]:
len1 = len(L1.vec.FV().NumPy())
L1.vec.FV().NumPy()[:] = np.zeros(len1)

In [ ]:
hel